In [1]:
#import the libraries
import math
import pandas_datareader as web
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import joblib
import pickle
plt.style.use('fivethirtyeight')

In [2]:
#MACRO DEF

STOCK = 'GOOG'
EPOCHS = 100
model_path = 'machine learning model/'+STOCK+'_model.h5'
error_model_path = "machine learning model/"+STOCK+"_error_model.h5"
error_data = "machine learning model/"+STOCK+"_error_data.csv"
START_DATE = '2012-01-01'
END_DATE = '2022-03-08'
PREDICT_DATE = '2022-03-10'
ERR_EPOCHS = 120

In [3]:
#IMPORTING THE MODEL
model = load_model(model_path)


In [4]:
#DEFINING SCALER
scaler = MinMaxScaler(feature_range=(0,1))

In [5]:
#get the Quote
stock_quote = web.DataReader(STOCK, data_source = 'yahoo', start=START_DATE, end=END_DATE)
#create new data frame
new_df = stock_quote.filter(['Close'])
#get the last 60 days closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#scale the data to be values between 0 and 1
last_60_days_scaled = scaler.fit_transform(last_60_days)
#create the empty list
x_test = []
#append the past 60 days
x_test.append(last_60_days_scaled)
#convert the x_test data set to a numpy array
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
#get the predicted scaled price
pred_price = model.predict(x_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)


[[2553.7842]]


In [6]:
apple_quote2 = web.DataReader(STOCK, data_source = 'yahoo', start=START_DATE, end=PREDICT_DATE)
print(apple_quote2['Close'])

Date
2012-01-03     331.462585
2012-01-04     332.892242
2012-01-05     328.274536
2012-01-06     323.796326
2012-01-09     310.067780
                 ...     
2022-03-04    2642.439941
2022-03-07    2529.290039
2022-03-08    2545.570068
2022-03-09    2677.320068
2022-03-10    2648.729980
Name: Close, Length: 2564, dtype: float64


In [7]:
#IMPORTING THE error MODEL
error_model = load_model(error_model_path)

dfnperr = pd.read_csv(error_data, sep=",")

dfnperr.shape

(512, 2)

In [8]:
#DEFINING SCALER
errscaler = MinMaxScaler(feature_range=(0,1))

In [9]:
#predict the error

newdf = dfnperr.filter(['Error'])

last_60_days_error = newdf[-60:].values


#scale the data to be values between 0 and 1
last_60_days_scaled_error = errscaler.fit_transform(last_60_days_error)

#create the empty list
x_errtest=[]

#append the past 60 days 
x_errtest.append(last_60_days_scaled_error)

#convert the x_test data set to a numpy array
x_errtest = np.array(x_errtest)
x_errtest = np.reshape(x_errtest,(x_errtest.shape[0],x_errtest.shape[1],1))

pred_error = error_model.predict(x_errtest)

#undo the scaling
pred_error = errscaler.inverse_transform(pred_error)

print(pred_error)

dfnperr.tail()

[[-4.256888]]


,Unnamed: 0,Error
507,507,-17.699463
508,508,-19.369629
509,509,62.985840
510,510,-103.729980
511,511,-78.492432


In [10]:
print("the predicted value with 1D error correction ")
print(pred_price[0][0]-abs(pred_error[0][0])*0.59)

the predicted value with 1D error correction 
2551.2726158189776
